In [ ]:
# Image_Classifier.ipynb

# To run this notebook, you'll need to have Jupyter Notebook installed.
# You can install it using: pip install jupyter

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Set the path to your dataset and the number of classes
train_data_dir = 'path/to/training_data'
num_classes = 2

# Set hyperparameters
batch_size = 32
epochs = 10

# Data augmentation to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

# Build a simple CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs)

# Save the trained model
model.save('image_classifier_model.h5')

# Load the trained model
model = load_model('image_classifier_model.h5')

# Load and preprocess the image for prediction
img_path = 'path/to/your/image.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale to the same scale used during training

# Make predictions
predictions = model.predict(img_array)

# Print the prediction results
class_names = ['class1', 'class2']  # Replace with your class labels
predicted_class = np.argmax(predictions, axis=1)
predicted_label = class_names[predicted_class[0]]
print(f'Predicted class: {predicted_label}')
